In [1]:
import os 
import numpy as np
import pandas as pd
import librosa
import pyworld
import time
import shutil
import matplotlib.pyplot as plt

from tools import *
from model import *

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
data_dir = "../data/NTT_corevo"
figure_dir = "../figure/NTT_corevo/Classifier"
model_dir = "../model/NTT_corevo/Classifier"
model_name = [
    "Classifier_lr3-4_e10000_b4_label0",
    "Classifier_lr3-4_e10000_b4_label1",
    "Classifier_lr3-4_e10000_b4_label2",
    "Classifier_lr3-4_e10000_b4_label3",
    "Classifier_lr3-4_e10000_b4_label4",
    "Classifier_lr3-4_e10000_b4_label5",
    "Classifier_lr3-4_e10000_b4_child",
    "Classifier_lr3-4_e10000_b4_adult",
    "Classifier_lr3-4_e10000_b4_elder",
    "Classifier_lr3-4_e10000_b4_male",
    "Classifier_lr3-4_e10000_b4_female",
    
]
model_dir_vae = "../model/NTT_corevo/VAE"
model_name_vae = "VAE_lr3-4_e10000_b4"

In [3]:
seed_value = 0
np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [4]:
sampling_rate = 16000
num_mcep = 36
frame_period = 5.0
n_frames = 512
label_num = 6

In [5]:
def data_load(batch_size = 1, label = -1):
    data_list = []
    label_list = []
    
    if (label == -1):
        random_label = True 
    else:
        random_label =  False
        
    for i in range(batch_size):
        
        if random_label :
            label = np.random.randint(0, label_num)
            
        sample_data_dir = os.path.join(data_dir, "labeled/{:02}".format(label))
        file = np.random.choice(os.listdir(sample_data_dir))
        
        frames = 0
        count = 0
        while frames < n_frames:

            wav, _ = librosa.load(os.path.join(sample_data_dir, file), sr = sampling_rate, mono = True)
            wav = librosa.util.normalize(wav, norm=np.inf, axis=None)
            wav = wav_padding(wav = wav, sr = sampling_rate, frame_period = frame_period, multiple = 4)
            f0, timeaxis, sp, ap, mc = world_decompose(wav = wav, fs = sampling_rate, frame_period = frame_period, num_mcep = num_mcep)

            if (count == 0):
                mc_transposed = np.array(mc).T
            else:
                mc_transposed = np.concatenate([mc_transposed, np.array(mc).T], axis =1)
            frames = np.shape(mc_transposed)[1]

            mean = np.mean(mc_transposed)
            std = np.std(mc_transposed)
            mc_norm = (mc_transposed - mean)/std

            count += 1

        start_ = np.random.randint(frames - n_frames + 1)
        end_ = start_ + n_frames
        
        data_list.append(mc_norm[:,start_:end_])
        label_list.append(label)

    return torch.Tensor(data_list).view(batch_size, 1, num_mcep, n_frames), torch.Tensor(label_list).view(batch_size, 1)


In [6]:
def save_figure(losses_list):
    if not os.path.exists(figure_dir):
            os.makedirs(figure_dir)
    losses_list = np.array(losses_list)
    plt.figure()
    for i in range(len(losses_list)):
        losses = losses_list[i]
        x = np.linspace(0, len(losses), len(losses))
        plt.plot(x, losses, label="label : {}".format(i))
        plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(figure_dir + "/" + "_result.png")

In [7]:
def model_save(model, label):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(model.state_dict(), os.path.join(model_dir, model_name[label]))
    
def model_load(label):
    model = Classifier()
    model.load_state_dict(torch.load(os.path.join(model_dir, model_name[label])))
    return model
    
def model_load_VAE():
    model = VAE()
    model.load_state_dict(torch.load(os.path.join(model_dir_vae, model_name_vae)))
    return model

In [8]:
learning_rate = 1e-3
learning_rate_ = 1e-4
num_epoch = 10000
batch_size = 4

num_label = 6

In [ ]:
model_vae = model_load_VAE()
model_vae.eval()

VAE(
  (conv1): Conv2d(1, 8, kernel_size=(3, 9), stride=(1, 1), padding=(1, 4))
  (conv1_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_gated): Conv2d(1, 8, kernel_size=(3, 9), stride=(1, 1), padding=(1, 4))
  (conv1_gated_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_sigmoid): Sigmoid()
  (conv2): Conv2d(8, 16, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
  (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_gated): Conv2d(8, 16, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
  (conv2_gated_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_sigmoid): Sigmoid()
  (conv3): Conv2d(16, 16, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
  (conv3_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3_gated): Conv2d(16, 16, kernel_size=(4, 8), stride=(2, 2),

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

losses_list = []

for label in range(num_label):

    model = Classifier().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    losses = []

    for epoch in range(num_epoch):
        epoch += 1
        lr = learning_rate_ *(1. / num_epoch) * (epoch) + learning_rate * (1. / num_epoch) * (num_epoch - epoch)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
            
        x_, label_ = data_load(batch_size)
        z_ = model_vae.predict(x_)
        
        optimizer.zero_grad()
        loss = model.calc_loss(z_, label, label_)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()

        print("Label {}  :  Epoch {}  :  Loss  {:.05}  :  LR {:.05}". format(label, epoch, loss.item(), lr))

    losses_list.append(losses)
    save_figure(losses_list)
    
    model_save(model, label)


cuda
Label 0  :  Epoch 1  :  Loss  5.724693775177002
Label 0  :  Epoch 2  :  Loss  5.652629852294922
Label 0  :  Epoch 3  :  Loss  5.459183692932129
Label 0  :  Epoch 4  :  Loss  5.738085746765137
Label 0  :  Epoch 5  :  Loss  5.487727165222168
Label 0  :  Epoch 6  :  Loss  5.4808173179626465
Label 0  :  Epoch 7  :  Loss  5.487957954406738
Label 0  :  Epoch 8  :  Loss  5.444278240203857
Label 0  :  Epoch 9  :  Loss  5.352506637573242
Label 0  :  Epoch 10  :  Loss  5.569849491119385


## the kind of classifiers
- label (1~6)
- female or male
- chilld or adult or elder

In [ ]:
losses_list = []

for episode in range(5):
    
    # 0 : child
    # 1 : adult
    # 2 : elder
    # 3 : male
    # 4 : female

    model = Classifier().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    losses = []

    for epoch in range(num_epoch):
        epoch += 1
        lr = learning_rate_ *(1. / num_epoch) * (epoch) + learning_rate * (1. / num_epoch) * (num_epoch - epoch)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
            
        x_, label_ = data_load(batch_size)
        z_ = model_vae.predict(x_)
        for i in range(batch_size):
            # 0 : "FE_CH"
            # 1 : "FE_AD"
            # 2 : "FE_EL"
            # 3 : "MA_CH"
            # 4 : "MA_AD"
            # 5 : "MA_EL"
            if episode == 0:
                if label_[i] == 0 or label_[i] == 3:
                    label_[i] = 0
                else:
                    label_[i] = 1
            elif episode == 1:
                if label_[i] == 1 or label_[i] == 4:
                    label_[i] = 0
                else:
                    label_[i] = 1
            elif episode == 2:
                if label_[i] == 2 or label_[i] == 5:
                    label_[i] = 0
                else:
                    label_[i] = 1
            elif episode == 3:
                if label_ [i]== 3 or label_[i] == 4 or label_[i] == 5:
                    label_[i] = 0
                else:
                    label_[i] = 1
            elif episode == 4:
                if label_[i] == 0 or label_[i] == 1 or label_[i] == 2:
                    label_[i] = 0
                else:
                    label_[i] = 1
                    
        optimizer.zero_grad()
        loss = model.calc_loss(z_, 0, label_)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()

        print("Label {}  :  Epoch {}  :  Loss  {:.05}  :  LR {:.05}". format(label, epoch, loss.item(), lr))

    losses_list.append(losses)
    save_figure(losses_list)
    
    model_save(model, episode+6)